In [ ]:
!pip install hf_xet
!pip install trl
!pip install --upgrade transformers
!pip install tensorboard
!pip install -U bitsandbytes
!pip install transformers_stream_generator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 38.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for transformers_stream_generator: filename=transformers_stream_generator-0.0.5-py3-none-any.whl size=12426 sha256=990be731c0fb9c3f27b4d2825e19862d3dbe997f58ed2fc77a60a6e97ed5dc74
  Stored in directory: /root/.cache/pip/wheels/23/e8/f0/b3c58c12d1ffe60bcc8c7d121115f26b2c1878653edfca48db
Successfully built transformers_stream_generator


In [ ]:
import os
from datasets import load_dataset, concatenate_datasets, Dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer, SFTConfig

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/SMU_MITB_LLM/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/SMU_MITB_LLM


In [ ]:
# load tuning sets for <Sunrise on the Reaping>
sunrise_train_path = './reasoned_qa_output/sunrise_train_data.json'
sunrise_val_path = './reasoned_qa_output/sunrise_val_data.json'
sunrise_test_path = './reasoned_qa_output/sunrise_test_data.json'

sunrise_train_datasets = Dataset.from_json(sunrise_train_path)
sunrise_val_datasets = Dataset.from_json(sunrise_val_path)
sunrise_test_datasets = Dataset.from_json(sunrise_test_path)

In [ ]:
# load tuning sets for <All Fours>
allfours_train_path = './reasoned_qa_output/allfours_train_data.json'
allfours_val_path = './reasoned_qa_output/allfours_val_data.json'
allfours_test_path = './reasoned_qa_output/allfours_test_data.json'

allfours_train_datasets = Dataset.from_json(allfours_train_path)
allfours_val_datasets = Dataset.from_json(allfours_val_path)
allfours_test_datasets = Dataset.from_json(allfours_test_path)

In [ ]:
# Combine two books together
train_set = concatenate_datasets([sunrise_train_datasets, allfours_train_datasets])
val_set = concatenate_datasets([sunrise_val_datasets, allfours_val_datasets])
test_set = concatenate_datasets([sunrise_test_datasets, allfours_test_datasets])

In [ ]:
# train_sample_size = train_set.num_rows // 5
# val_sample_size = val_set.num_rows // 5
# test_sample_size = test_set.num_rows // 5

In [ ]:
# train_set = train_set.shuffle(seed=42).select(range(train_sample_size))
# val_set = val_set.shuffle(seed=42).select(range(val_sample_size))
# test_set = test_set.shuffle(seed=42).select(range(test_sample_size))

In [ ]:
train_set

Dataset({
    features: ['Question', 'Answer', 'Reasoning'],
    num_rows: 958
})

In [ ]:
val_set

Dataset({
    features: ['Question', 'Answer', 'Reasoning'],
    num_rows: 120
})

In [ ]:
test_set

Dataset({
    features: ['Question', 'Answer', 'Reasoning'],
    num_rows: 120
})

In [ ]:
print(train_set["Question"][-1])

What trick does Jordi teach the protagonist to help stop thinking about Davey?


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Load Reasoning LLM
gemma_model = 'google/gemma-2b'
# Load the specified tokenizer for the model to ensure consistency
tokenizer = AutoTokenizer.from_pretrained(gemma_model)

In [ ]:
# load model
model = AutoModelForCausalLM.from_pretrained(
    gemma_model,
    device_map="auto"  # instructs the transformers library to automatically distribute the model's layer cross available decives to optimise memory usage
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 19,611,648 || all params: 2,525,784,064 || trainable%: 0.7765


In [ ]:
def formatting_prompts_func(examples):
    texts = []
    for question, answer, reasoning_list in zip(examples["Question"], examples["Answer"], examples["Reasoning"]):
        reasoning_str = "\n".join([f"Step {i+1}: {step}" for i, step in enumerate(reasoning_list)])
        formatted_text = (
            f"### Question:\n{question}\n\n"
            f"### Thought Process:\n"
            f"{reasoning_str}\n\n"
            f"### Answer:\n{answer}"
        )
        texts.append(formatted_text)
    return {"text": texts}

In [ ]:
train_dataset_formatted = train_set.map(
    formatting_prompts_func,
    batched=True,
    remove_columns=train_set.column_names
)
validation_dataset_formatted = val_set.map(
    formatting_prompts_func,
    batched=True,
    remove_columns=val_set.column_names
)

Map:   0%|          | 0/958 [00:00<?, ? examples/s]

Map:   0%|          | 0/120 [00:00<?, ? examples/s]

In [ ]:
print(train_dataset_formatted['text'][-1])

### Question:
What trick does Jordi teach the protagonist to help stop thinking about Davey?

### Thought Process:
Step 1: During their meeting, Jordi demonstrates a technique she used to quit smoking.
Step 2: The protagonist snaps a rubber band on her wrist each time she thinks of Davey.
Step 3: This method aims to reduce obsessive thoughts through punishment.

### Answer:
Jordi teaches her to use a rubber band for negative reinforcement whenever she thinks about him.


In [ ]:
import inspect
from trl import SFTTrainer
print(inspect.signature(SFTTrainer.__init__))

(self, model: Union[str, torch.nn.modules.module.Module, transformers.modeling_utils.PreTrainedModel], args: Union[trl.trainer.sft_config.SFTConfig, transformers.training_args.TrainingArguments, NoneType] = None, data_collator: Optional[transformers.data.data_collator.DataCollator] = None, train_dataset: Union[datasets.arrow_dataset.Dataset, datasets.iterable_dataset.IterableDataset, NoneType] = None, eval_dataset: Union[datasets.arrow_dataset.Dataset, dict[str, datasets.arrow_dataset.Dataset], NoneType] = None, processing_class: Union[transformers.tokenization_utils_base.PreTrainedTokenizerBase, transformers.image_processing_utils.BaseImageProcessor, transformers.feature_extraction_utils.FeatureExtractionMixin, transformers.processing_utils.ProcessorMixin, NoneType] = None, compute_loss_func: Optional[Callable] = None, compute_metrics: Optional[Callable[[transformers.trainer_utils.EvalPrediction], dict]] = None, callbacks: Optional[list[transformers.trainer_callback.TrainerCallback]] 

In [ ]:
# SFTConfig 应该包含原 TrainingArguments 的参数，以及 SFT 特有的参数。
sft_config = SFTConfig(
    output_dir="./results", # From TrainingArguments
    num_train_epochs=3, # From TrainingArguments
    per_device_train_batch_size=4, # From TrainingArguments
    per_device_eval_batch_size=4,
    warmup_steps=500,
    gradient_accumulation_steps=1, # From TrainingArguments
    learning_rate=2e-4, # From TrainingArguments
    logging_steps=10, # From TrainingArguments
    save_steps=500, # From TrainingArguments
    # Add SFT-specific parameters here:
    max_seq_length=128, # Now correctly placed in SFTConfig
    dataset_text_field="text", # Important for SFTTrainer
    packing=True, # Recommended for efficient SFT
    optim="adafactor",
    fp16=True, # For mixed precision, if SFTConfig handles this
)

# 注意：现在 SFTTrainer 只需要一个 `args` 参数，就是您的 sft_config
trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer, # Keep this, as it's separate from args
    train_dataset=train_dataset_formatted,
    eval_dataset=validation_dataset_formatted,
    peft_config=lora_config,
    args=sft_config, # <-- Pass the SFTConfig object here!
    # formatting_func=formatting_func, # If you have a custom formatting function
)

average_tokens_across_devices is set to True but it is invalid when world size is1. Turn it to False automatically.
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:404: UserWarning: Padding-free training is enabled, but the attention implementation is not set to 'flash_attention_2'. Padding-free training flattens batches into a single sequence, and 'flash_attention_2' is the only known attention mechanism that reliably supports this. Using other implementations may lead to unexpected behavior. To ensure compatibility, set `attn_implementation='flash_attention_2'` in the model configuration, or verify that your attention mechanism can handle flattened sequences.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:450: UserWarning: You are using packing, but the attention implementation is not set to 'flash_attention_2'. Packing flattens batches into a single sequence, and 'flash_attention_2' is the only known attention mechanism that re

Adding EOS to train dataset:   0%|          | 0/958 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/958 [00:00<?, ? examples/s]

Packing train dataset:   0%|          | 0/958 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/120 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/120 [00:00<?, ? examples/s]

Packing eval dataset:   0%|          | 0/120 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
print("Model Tuning Start...")
trainer.train()
print("Model Tuning Completed")

Model Tuning Start...


Step,Training Loss
10,2.804200
20,2.680800
30,2.624800
40,2.502200
50,2.485500
60,2.393600
70,2.220300
80,2.106200
90,2.052800
100,1.859400


Model Tuning Completed


In [ ]:
# Save LoAR Adapter
output_model_path = "./LoRA_Gemma"
trainer.save_model(output_model_path)
print(f"Fine-tuned LoRA LoRA Adapter saved to: {output_model_path}")

Fine-tuned LoRA LoRA Adapter saved to: ./LoRA_Gemma


In [50]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig
gemma_model = 'google/gemma-2b'

base_model_name_or_path = gemma_model
lora_adapter_path = "./LoRA_Gemma"

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

print(f"Loading base model from {base_model_name_or_path}...")
model = AutoModelForCausalLM.from_pretrained(
    base_model_name_or_path,
    torch_dtype=torch.bfloat16 if device == "cuda" else torch.float32,
    device_map="auto"
)

print(f"Loading tokenizer from {base_model_name_or_path}...")
tokenizer = AutoTokenizer.from_pretrained(base_model_name_or_path)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"Loading LoRA adapter from {lora_adapter_path} and merging...")
peft_config = PeftConfig.from_pretrained(lora_adapter_path)

model = PeftModel.from_pretrained(model, lora_adapter_path)
print("LoRA adapter loaded successfully.")
model.eval()

Using device: cuda
Loading base model from google/gemma-2b...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading tokenizer from google/gemma-2b...
Loading LoRA adapter from ./LoRA_Gemma and merging...
LoRA adapter loaded successfully.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GemmaForCausalLM(
      (model): GemmaModel(
        (embed_tokens): Embedding(256000, 2048, padding_idx=0)
        (layers): ModuleList(
          (0-17): 18 x GemmaDecoderLayer(
            (self_attn): GemmaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_pro